In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import datetime
from time import sleep

In [ ]:
KEY = ''
PAGE_NOT_FOUND = '<title>\nScopus preview -  \nScopus - Page not found\n</title>'
GATEWAY_TIME_OUT = '<title>www.scopus.com | 504: Gateway time-out</title>'
BATCH_COUNTER = range(0, 4000, 25)

df = pd.DataFrame(columns=('titles', 'pub_dates', 'abstracts', 'author_keywords', 'links'))
s = requests.session()

In [ ]:
for batch_counter in BATCH_COUNTER:
    
    while True:
        try:
            response = s.get('https://api.elsevier.com/content/search/scopus',
                             params={'apiKey': KEY,
                             'query': 'PUBYEAR = 2011',
                             'start': batch_counter})
        except requests.exceptions.ConnectionError:
            print('Sleep 1 seconds to recieve new batch')
            sleep(1)
            continue
        break
        
    parsed_dict = json.loads(response.text)

    titles = [i['dc:title'] for i in parsed_dict['search-results']['entry']]
    pub_dates = [i['prism:coverDate'] for i in parsed_dict['search-results']['entry']]
    preview_links = [i['link'][2]['@href'] for i in parsed_dict['search-results']['entry']]

    abstracts, author_keywords, pages = [], [], []

    for i in preview_links:

        while True:
            try:
                response = s.get(i, headers={'User-Agent': UserAgent().Chrome})
            except requests.exceptions.ConnectionError:
                print('Sleep 1 second to recieve new article')
                sleep(1)
                continue
            else:
                soup = BeautifulSoup(response.content,'html.parser')
                if str(soup.find('title')) == PAGE_NOT_FOUND:
                    print('Page not found error. Wait 5 seconds')
                    sleep(5)
                    continue
                elif str(soup.find('title')) == GATEWAY_TIME_OUT:
                    print('Gateway time-out error. Wait 5 seconds')
                    sleep(5)
                    continue
                else:
                    break
        
        pages.append(soup)
        
        try:
            abstract = soup.find('section', attrs={'class': 'row', 'id': 'abstractSection'})
            abstract = abstract.find('p').contents[0]
            abstracts.append(abstract)
        except:
            author_keyword.append('[No abstract available]')

        try:
            author_keyword = soup.find('section', attrs={'id': 'authorKeywords'})
            author_keyword = author_keyword.find_all('span')
            author_keyword = [i.contents[0] for i in author_keyword]
            author_keywords.append(author_keyword)
        except:
            author_keywords.append(list(['No keywords available']))

    temp_df = pd.DataFrame({'titles': titles, 'pub_dates': pub_dates, 'abstracts': abstracts, \
                            'author_keywords': author_keywords, 'links': preview_links})
    df = pd.concat([df, temp_df], axis=0)
    
    batch_counter += 25
    print('batch_counter =', str(batch_counter), ', now =', datetime.datetime.now())

print('4000 строк смайнено успешно')

In [ ]:
preview_links

In [ ]:
df = df.reset_index()
df = main_df.drop('index', axis=1)

In [ ]:
df.to_parquet('mined.parquet', engine='pyarrow')